In [35]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import re
import tensorflow as tf

In [26]:
def display_images(images, n_line, l_col):
    plt.figure(figsize=(10, 10))
    for i in range(n_line*l_col):
        ax = plt.subplot(n_line, l_col, i + 1)
        plt.imshow(images[i])
        plt.axis("off")

In [27]:
def build_boxes(x, y, w, h):
    N = 5
    shift = list(range(-N, N+1, 2))
    
    pos = []
    for i in shift:
        new_x = x + i
        
        for j in shift:
            new_y = y + j
            
            new_pos = (new_x, new_y, w, h)
            pos.append(new_pos)
            
            # X 1.2
            zoom = [1.2, 0.8]
            for z in zoom:
                new_w, new_h = int(w*z), int(h*z)
                new_x = new_x - (abs(new_w - w) // 2)
                new_y = new_y - (abs(new_h - h) // 2)
                
                pos.append((new_x, new_y, new_w, new_h))
            
    return pos

In [28]:
def read_bounding_box(path):
    boxes = {}
    m = re.search('VIDEOS/(.+)\.mp4', path)
    filename = m.group(1)
    
    f = open(f'../GT/{filename}_2_bboxes.txt', 'r')
    lines = f.readlines()
    
    for l in lines:
        elmts = l.split(' ')
        frame = int(elmts[0])
        
        if elmts[1] == '1':
            boxes[frame] = (int(elmts[2]), int(elmts[3]), int(elmts[4]), int(elmts[5]))
            
    return boxes

In [36]:
model = tf.keras.models.load_model('../models/custom_13_12_2020', custom_objects=None, compile=True, options=None)

In [59]:
def get_images(filename):
    boxes = read_bounding_box(filename)
    
    cap = cv2.VideoCapture(filename)
    nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    if(not cap.isOpened()):
        print("ERROR: unable to read video:", video_filename)
        sys.exit()
        
    print(f'Loading {nframes} frames')
    print(len(boxes))
    
    foundFirst = False
    for f in range(nframes):
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        if f in boxes:
            if not foundFirst:
                box = boxes[f]
            else:
                print('a')
                
                # box = boxes[f]
                x, y, w, h = box

                patch = frame[y:y+h, x:x+w]
                new_boxes = build_boxes(x, y, w, h)

                patches = []
                for bb in new_boxes:
                    x1, y1, w1, h1 = bb
                    new_patch = tf.expand_dims(frame[y1:y1+h1, x1:x1+w1, :], 0) 
                    patches.append(tf.image.resize(new_patch, (227, 227)))

                pred = model.predict(patches)

                # on récupère la meilleure pred
                # on dit box == meilleure pred
                if f == 100:
                    display_images(patches, 9, 9)
            foundFirst = True


In [ ]:
get_images('../VIDEOS/CanOfCocaColaPlace6Subject4.mp4')

Loading 186 frames
163
a
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78

a
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1

a
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1

a
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1

a
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1

a
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1, 189, 117, 3)
(1, 126, 78, 3)
(1, 158, 98, 3)
(1

In [ ]:
boxes